In [2]:
# 7.3.5 - 커스텀 체인 (Custom Chain)
# LangChain의 기본 체인으로 해결되지 않는 특정 작업을 위한 사용자 정의 체인 만들기

# 필요한 라이브러리 설치
# !pip install -U langchain==0.2.17 langchain-openai langchain-core

# =============================================================================
# API 키 설정 (독자용)
# =============================================================================
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    if api_key:
        os.environ['OPENAI_API_KEY'] = api_key
        print("API 키가 설정되었습니다!")
else:
    print("기존 환경 변수의 API 키를 사용합니다.")

print("=" * 80)
print("7.3.5 커스텀 체인 (Custom Chain)")
print("=" * 80)

print("""
원서 코드의 문제점:
1. Chain 클래스와 LLMChain이 deprecated됨
2. run() 메서드가 deprecated됨

수정 내용:
- Chain 클래스 → RunnableSequence와 커스텀 함수 조합
- LLMChain → prompt | llm 방식
- run() → invoke() 메서드

새로운 접근법:
- RunnableLambda를 사용한 커스텀 로직
- 함수형 체인 구성
- | 연산자를 활용한 체인 연결
""")

# =============================================================================
# 라이브러리 import
# =============================================================================
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from typing import Dict, Any

# LLM 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# =============================================================================
# 방법 1: RunnableLambda를 사용한 커스텀 체인 (최신 방식)
# =============================================================================
print("\n" + "=" * 60)
print("방법 1: RunnableLambda를 사용한 현대적 커스텀 체인")
print("=" * 60)

# 프롬프트 정의
prompt1 = PromptTemplate.from_template("What is the meaning of the word '{word}'?")
prompt2 = PromptTemplate.from_template("Suggest a synonym for the word '{word}'.")

# 개별 체인 구성
chain1 = prompt1 | llm
chain2 = prompt2 | llm

# 커스텀 연결 함수
def concatenate_outputs(inputs: Dict[str, Any]) -> Dict[str, str]:
    """두 체인의 출력을 연결하는 함수"""
    word = inputs["word"]
    
    # 각 체인 실행
    output1 = chain1.invoke({"word": word})
    output2 = chain2.invoke({"word": word})
    
    # 결과 연결
    concatenated = f"정의: {output1.content.strip()}\n\n동의어: {output2.content.strip()}"
    
    return {"concat_output": concatenated}

# RunnableLambda로 커스텀 체인 생성
modern_concat_chain = RunnableLambda(concatenate_outputs)

print("테스트 단어: 'artificial'")

try:
    result = modern_concat_chain.invoke({"word": "artificial"})
    print("\n연결된 출력:")
    print(result["concat_output"])
    
except Exception as e:
    print(f"오류 발생: {e}")

# =============================================================================
# 방법 2: 더 복잡한 커스텀 체인 예제
# =============================================================================
print("\n" + "=" * 60)
print("방법 2: 다단계 분석 커스텀 체인")
print("=" * 60)

# 여러 단계의 분석을 수행하는 커스텀 체인
def multi_analysis_chain(inputs: Dict[str, Any]) -> Dict[str, Any]:
    """단어에 대한 다각도 분석을 수행하는 체인"""
    word = inputs["word"]
    
    # 각각의 분석 프롬프트
    prompts = {
        "definition": PromptTemplate.from_template("Define the word '{word}' in simple terms."),
        "etymology": PromptTemplate.from_template("What is the etymology of the word '{word}'?"),
        "usage": PromptTemplate.from_template("Give an example sentence using the word '{word}'."),
        "synonyms": PromptTemplate.from_template("List 3 synonyms for the word '{word}'.")
    }
    
    results = {}
    
    # 각 분석 실행
    for analysis_type, prompt in prompts.items():
        chain = prompt | llm
        response = chain.invoke({"word": word})
        results[analysis_type] = response.content.strip()
    
    # 결과 정리
    formatted_output = f"""단어 분석: {word}

📖 정의: {results['definition']}

🌍 어원: {results['etymology']}

📝 사용 예시: {results['usage']}

🔄 동의어: {results['synonyms']}"""
    
    return {"analysis": formatted_output}

# 다단계 분석 체인 생성
analysis_chain = RunnableLambda(multi_analysis_chain)

print("다단계 분석 테스트: 'intelligence'")

try:
    analysis_result = analysis_chain.invoke({"word": "intelligence"})
    print("\n다각도 분석 결과:")
    print(analysis_result["analysis"])
    
except Exception as e:
    print(f"다단계 분석 오류: {e}")

# =============================================================================
# 방법 3: 조건부 로직이 있는 커스텀 체인
# =============================================================================
print("\n" + "=" * 60)
print("방법 3: 조건부 로직이 있는 스마트 체인")
print("=" * 60)

def smart_word_processor(inputs: Dict[str, Any]) -> Dict[str, Any]:
    """입력 단어의 특성에 따라 다른 처리를 하는 스마트 체인"""
    word = inputs["word"]
    word_length = len(word)
    
    if word_length <= 4:
        # 짧은 단어: 간단한 정의
        prompt = PromptTemplate.from_template("Give a very brief definition of '{word}'.")
        approach = "간단 정의"
    elif word_length <= 8:
        # 중간 길이: 정의 + 예시
        prompt = PromptTemplate.from_template("Define '{word}' and give one example sentence.")
        approach = "정의 + 예시"
    else:
        # 긴 단어: 상세 분석
        prompt = PromptTemplate.from_template("Provide a detailed explanation of '{word}' including its origin and usage.")
        approach = "상세 분석"
    
    # 체인 실행
    chain = prompt | llm
    response = chain.invoke({"word": word})
    
    result = f"처리 방식: {approach}\n단어 길이: {word_length}자\n\n결과:\n{response.content}"
    
    return {"smart_output": result}

# 스마트 처리 체인 생성
smart_chain = RunnableLambda(smart_word_processor)

# 다양한 길이의 단어로 테스트
test_words = ["AI", "computer", "artificial"]

print("다양한 길이의 단어 처리 테스트:")

try:
    for word in test_words:
        result = smart_chain.invoke({"word": word})
        print(f"\n테스트 단어: '{word}'")
        print(result["smart_output"])
        print("-" * 50)
        
except Exception as e:
    print(f"스마트 체인 오류: {e}")

# =============================================================================
# 방법 4: 체인들의 체인 (체인 조합)
# =============================================================================
print("\n" + "=" * 60)
print("방법 4: 체인들의 체인 - 복합 워크플로우")
print("=" * 60)

def workflow_chain(inputs: Dict[str, Any]) -> Dict[str, Any]:
    """여러 체인을 순차적으로 실행하는 워크플로우"""
    word = inputs["word"]
    
    # 1단계: 기본 정의
    definition_prompt = PromptTemplate.from_template("Define '{word}' in one sentence.")
    definition_chain = definition_prompt | llm
    definition = definition_chain.invoke({"word": word}).content.strip()
    
    # 2단계: 정의를 바탕으로 카테고리 분류
    category_prompt = PromptTemplate.from_template(
        "Based on this definition: '{definition}', what category does the word '{word}' belong to? (e.g., technology, nature, emotion, etc.)"
    )
    category_chain = category_prompt | llm
    category = category_chain.invoke({"word": word, "definition": definition}).content.strip()
    
    # 3단계: 카테고리에 맞는 관련 단어 제안
    related_prompt = PromptTemplate.from_template(
        "Suggest 3 words related to '{word}' in the '{category}' category."
    )
    related_chain = related_prompt | llm
    related_words = related_chain.invoke({"word": word, "category": category}).content.strip()
    
    # 최종 결과 구성
    workflow_result = f"""워크플로우 결과:

1️⃣ 단어: {word}
2️⃣ 정의: {definition}
3️⃣ 카테고리: {category}
4️⃣ 관련 단어: {related_words}"""
    
    return {"workflow_output": workflow_result}

# 워크플로우 체인 생성
workflow = RunnableLambda(workflow_chain)

print("워크플로우 테스트: 'robot'")

try:
    workflow_result = workflow.invoke({"word": "robot"})
    print("\n워크플로우 실행 결과:")
    print(workflow_result["workflow_output"])
    
except Exception as e:
    print(f"워크플로우 오류: {e}")

# =============================================================================
# 원서 방식과 비교 (참고용)
# =============================================================================
print("\n" + "=" * 60)
print("원서 방식 vs 새로운 방식 비교")
print("=" * 60)

print("""
원서 코드 (Deprecated):
from langchain.chains.base import Chain

class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain
    
    def _call(self, inputs):
        output1 = self.chain_1.run(inputs)    # ⚠️ Deprecated
        output2 = self.chain_2.run(inputs)    # ⚠️ Deprecated
        return {"concat_output": output1 + output2}

새로운 방식 (현재 권장):
from langchain_core.runnables import RunnableLambda

def concatenate_outputs(inputs):
    output1 = chain1.invoke(inputs)           # ✅ 권장
    output2 = chain2.invoke(inputs)           # ✅ 권장
    return {"concat_output": output1.content + output2.content}

custom_chain = RunnableLambda(concatenate_outputs)
""")

# =============================================================================
# 활용 팁과 베스트 프랙티스
# =============================================================================
print("\n" + "=" * 60)
print("커스텀 체인 활용 팁")
print("=" * 60)

print("""
커스텀 체인 설계 원칙:

1. 단순함 유지:
   - 복잡한 로직은 여러 개의 작은 함수로 분할
   - 각 함수는 하나의 명확한 역할 수행

2. 재사용성:
   - 공통 로직은 별도 함수로 분리
   - 매개변수를 통한 유연한 설정

3. 오류 처리:
   - try-except 블록으로 예외 상황 대비
   - 의미 있는 오류 메시지 제공

4. 테스트 가능성:
   - 각 단계별로 독립적 테스트 가능하도록 설계
   - 입력/출력 형식 표준화

5. 성능 최적화:
   - 불필요한 API 호출 최소화
   - 결과 캐싱 고려

실제 활용 사례:
- 다국어 번역 파이프라인
- 문서 요약 및 분석 시스템
- 고객 서비스 응답 생성기
- 창의적 콘텐츠 제작 도구
- 데이터 분석 및 보고서 생성
""")

print("\n🎉 7.3.5 커스텀 체인 예제 완료!")

기존 환경 변수의 API 키를 사용합니다.
7.3.5 커스텀 체인 (Custom Chain)

원서 코드의 문제점:
1. Chain 클래스와 LLMChain이 deprecated됨
2. run() 메서드가 deprecated됨

수정 내용:
- Chain 클래스 → RunnableSequence와 커스텀 함수 조합
- LLMChain → prompt | llm 방식
- run() → invoke() 메서드

새로운 접근법:
- RunnableLambda를 사용한 커스텀 로직
- 함수형 체인 구성
- | 연산자를 활용한 체인 연결


방법 1: RunnableLambda를 사용한 현대적 커스텀 체인
테스트 단어: 'artificial'

연결된 출력:
정의: The word 'artificial' refers to something that is made or produced by human beings rather than occurring naturally. It can also refer to something that is not genuine or authentic.

동의어: synthetic

방법 2: 다단계 분석 커스텀 체인
다단계 분석 테스트: 'intelligence'

다각도 분석 결과:
단어 분석: intelligence

📖 정의: Intelligence is the ability to learn, understand, and solve problems. It involves being able to think critically, make decisions, and adapt to new situations.

🌍 어원: The word "intelligence" comes from the Latin word "intelligentia," which is derived from the verb "intelligere," meaning "to understand" or "to comprehend." The prefix "inter-" m